 "1. GET SQUARE/RECTANGLE GROUND"
===============================

In [4]:
import cv2
import numpy as np
import glob
import os
import math

counter = 0
circles = np.zeros((4,2), np.int)
pathResultImages = r"Result_Images\\"

def mousePoints (event, x, y, flags, params):
    global counter
    #print('-event=', event,' -x,y:',x,y,'\t-flags:', flags, ' -params:', params)
    if event == cv2.EVENT_LBUTTONDOWN:
        #print(x,y)
        circles[counter] = x,y
        counter = counter + 1
        #print(circles)

def resizeImgCrop(img):
    print("==> resizeImgCrop")
    height = img.shape[0]
    width = img.shape[1]
    print("Height: ", height)
    print("Width: ", width)
    Ratio = 3
    newHeight = int(height/Ratio)
    newWidtht = int(width/Ratio)
    print("newHeight: ", newHeight)
    print("newWidtht: ", newWidtht)
    print()
    return newWidtht, newHeight

def getRealCoordinate(circles):
    leftTop = circles[0]
    rghtTop = circles[1]
    leftDwn = circles[2]
    rghtDwn = circles[3]
    return leftTop, rghtTop, leftDwn, rghtDwn

def getLengthWidthGround(leftTop, rghtTop, leftDwn, rghtDwn):
    # Length Top (left Top - right Top)
    lengthGroundTop = int( math.sqrt((abs(leftTop[0] - rghtTop[0]))**2 + (abs(leftTop[1] - rghtTop[1]))**2) )
    # Length Down (left Down - right Down)
    lengthGroundDown = int( math.sqrt((abs(leftDwn[0] - rghtDwn[0]))**2 + (abs(leftDwn[1] - rghtDwn[1]))**2) )
    # Width Left (left Top - left Down)
    widthGroundLeft = int( math.sqrt((abs(leftTop[0] - leftDwn[0]))**2 + (abs(leftTop[1] - leftDwn[1]))**2) )
    # Width Right (right Top - right Down)
    widthGroundRight = int( math.sqrt((abs(rghtTop[0] - rghtDwn[0]))**2 + (abs(rghtTop[1] - rghtDwn[1]))**2) )
    # =============== Find The Biggest
    if lengthGroundTop > lengthGroundDown:
        lengthGround = lengthGroundTop
    else:
        lengthGround = lengthGroundDown
    if widthGroundLeft > widthGroundRight:
        widthGround = widthGroundLeft
    else:
        widthGround = widthGroundRight
    return lengthGround, widthGround

def convertGround(leftTop,rghtTop,leftDwn,rghtDwn, lengthGround,widthGround):
    # Real Images
    pts1 = np.float32([leftTop,rghtTop,leftDwn,rghtDwn])
    # Destination Parameter Images (Rectangle)
    pts2 = np.float32([(0,0),(lengthGround,0),(0,widthGround),(lengthGround,widthGround)])
    matrix = cv2.getPerspectiveTransform(pts1,pts2)
    return matrix

def main():
    global counter
    #================================================
    # Crop Filename Images
    cropImageName = "BEV Camera Show (Crop).jpg"
    imgCrop = cv2.imread(cropImageName)
    # Get Parameter for Resize
    newWidtht, newHeight = resizeImgCrop(imgCrop)
    while True:
        # Resize
        imgCrop = cv2.resize(imgCrop, (newWidtht, newHeight))
        # Show
        cv2.imshow(cropImageName, imgCrop)
        # Get Coordinate to Get Square/Rectangle Ground
        for x in range(0,4):
            cv2.circle(imgCrop,(circles[x][0], circles[x][1]), 3, (0,255,0), cv2.FILLED)
        # Click Mouse
        cv2.setMouseCallback(cropImageName, mousePoints)
        if (cv2.waitKey(1) & 0xFF == ord('q')) or counter == 4:
            break
        cv2.waitKey(1)
    cv2.destroyAllWindows()
    counter = 0

    # Get Real Coordinate
    leftTopGround, rghtTopGround, leftDwnGround, rghtDwnGround = getRealCoordinate(circles)
    print("Get Real Coordinate: ", leftTopGround, rghtTopGround, leftDwnGround, rghtDwnGround)
    # Get Length and Width Ground
    lengthGround, widthGround = getLengthWidthGround(leftTopGround, rghtTopGround, leftDwnGround, rghtDwnGround)
    print("Get Length and Width Ground: ", lengthGround, widthGround)

    # ================== Changes Image to Rectangle
    Matrix = convertGround(leftTopGround,rghtTopGround,leftDwnGround,rghtDwnGround, lengthGround,widthGround)
    # Read Image
    imgCrop = cv2.imread(cropImageName)
    # Resize
    imgCrop = cv2.resize(imgCrop, (newWidtht, newHeight))
    imgGround = cv2.warpPerspective(imgCrop,Matrix,(lengthGround,widthGround))
    # Show Image
    cv2.imshow("-Convert-" + cropImageName, imgGround)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    # Write New Image
    cv2.imwrite(pathResultImages + "1.Rectangle-" + cropImageName, imgGround)
    
    # New Image Shape
    print()
    print("==> Size New Rectangle Image")
    newHeight = imgGround.shape[0]
    newWidth = imgGround.shape[1]
    print("Height: ", newHeight)
    print("Width: ", newWidth)


if __name__ == '__main__':       
    # Calling main() function 
    main()

==> resizeImgCrop
Height:  2125
Width:  3137
newHeight:  708
newWidtht:  1045

Get Real Coordinate:  [131   9] [961   2] [  3 652] [1042  704]
Get Length and Width Ground:  1040 706

==> Size New Rectangle Image
Height:  706
Width:  1040


==> resizeImgCrop
Height:  2125
Width:  3137
newHeight:  708
newWidtht:  1045

Get Real Coordinate:  [131   9] [961   2] [  3 652] [1042  704]
Get Length and Width Ground:  1040 706

==> Size New Rectangle Image
Height:  706
Width:  1040

"2. Convert Birds Eye View Camera A"
===================================

In [4]:
import cv2
import numpy as np
import glob
import os
import math

counter = 0
counterBEV = 0 
circlesConvert = np.zeros((16,2), np.int)
circlesBEV = np.zeros((12,2), np.int)
pathResultImages = r"Result_Images\\"

#>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
def readCamera(Camera_Filename):
    camera = cv2.imread(Camera_Filename)
    if (Camera_Filename[:8] == "Camera_A"):
        camera = cv2.rotate(camera, cv2.ROTATE_180)
        # Rotate A - No 180 Degree
    elif (Camera_Filename[:8] == "Camera_B"):
        # Rotate B - Rotate 90 COUNTERCLOCKWISE
        camera = cv2.rotate(camera, cv2.cv2.ROTATE_90_COUNTERCLOCKWISE)
    elif (Camera_Filename[:8] == "Camera_C"):
        # Camera C - Rotate 90 CLOCKWISE
        camera = cv2.rotate(camera, cv2.cv2.ROTATE_90_CLOCKWISE)
    elif (Camera_Filename[:8] == "Camera_D"):
        # Camera D - No Rotate 
        pass
    return camera
#>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>

#===================================================================================
def mousePointsBEV (event, x, y, flags, params):
    global counterBEV
    #print('-event=', event,' -x,y:',x,y,'\t-flags:', flags, ' -params:', params)
    if event == cv2.EVENT_LBUTTONDOWN:
        #print(x,y)
        circlesBEV[counterBEV] = x,y
        counterBEV = counterBEV + 1
        #print(circlesBEV)
def mousePoints (event, x, y, flags, params):
    global counter
    #print('-event=', event,' -x,y:',x,y,'\t-flags:', flags, ' -params:', params)
    if event == cv2.EVENT_LBUTTONDOWN:
        #print(x,y)
        circlesConvert[counter] = x,y
        counter = counter + 1
        #print(circlesConvert)
#===================================================================================

#===================================================================================
def getCoordinateCamera(imgCameraA,imgCameraB,imgCameraC,imgCameraD,  circlesConvert):
    for x in range(0,16):
        cv2.circle(imgCameraA,(circlesConvert[x][0], circlesConvert[x][1]), 3, (0,255,0), cv2.FILLED)
        cv2.circle(imgCameraB,(circlesConvert[x][0], circlesConvert[x][1]), 3, (0,255,0), cv2.FILLED)
        cv2.circle(imgCameraC,(circlesConvert[x][0], circlesConvert[x][1]), 3, (0,255,0), cv2.FILLED)
        cv2.circle(imgCameraD,(circlesConvert[x][0], circlesConvert[x][1]), 3, (0,255,0), cv2.FILLED)
def getCoordinateBEV(imgBEV, circlesBEV):
    for x in range(0,12):
        cv2.circle(imgBEV,(circlesBEV[x][0], circlesBEV[x][1]), 3, (0,255,0), cv2.FILLED)
#===================================================================================

#===================================================================================
def clickMouseCamera(Camera_A_Name,Camera_B_Name,Camera_C_Name,Camera_D_Name, mousePoints):
    cv2.setMouseCallback(Camera_A_Name, mousePoints)
    cv2.setMouseCallback(Camera_B_Name, mousePoints)
    cv2.setMouseCallback(Camera_C_Name, mousePoints)
    cv2.setMouseCallback(Camera_D_Name, mousePoints)
def clickMouseBEV(BEV_Name, mousePointsBEV):
    cv2.setMouseCallback(BEV_Name, mousePointsBEV)
#===================================================================================

#===================================================================================
def showLineSmallSquare(pointLeftTop, pointRightTop, pointLeftDwn, pointRightDwn, BEV_PathName):
    color = (0,0,255)
    thickness=2
    img = cv2.imread(BEV_PathName)
    cv2.line(img, tuple(pointLeftTop), tuple(pointRightTop), color, thickness)
    cv2.line(img, tuple(pointRightTop), tuple(pointRightDwn), color, thickness)
    cv2.line(img, tuple(pointRightDwn), tuple(pointLeftDwn), color, thickness)
    cv2.line(img, tuple(pointLeftDwn), tuple(pointLeftTop), color, thickness)
    cv2.imshow('Show Line Small Square', img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    return img
#===================================================================================

#>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
def convertBEV(Cnv1st,Cnv2nd,Cnv3rd,Cnv4th, BEV1st,BEV2nd,BEV3rd,BEV4th):
    # Real Images
    pts1 = np.float32([Cnv1st,Cnv2nd,Cnv3rd,Cnv4th])
    # Destination Parameter Images (BEV)
    pts2 = np.float32([BEV1st,BEV2nd,BEV3rd,BEV4th])
    matrix = cv2.getPerspectiveTransform(pts1,pts2)
    return matrix
#===================================================================================
def showBEV_Result(img,Camera_Filename,matrix, width,height):
    imgBEV = cv2.warpPerspective(img,matrix, (width,height))
    cv2.imshow("-BEV-" + Camera_Filename, imgBEV)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    return imgBEV
#>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>

def main():
    global counter
    global counterBEV
    #================================================
    # Bird's Eye View Filename Image
    BEV_Name = "1.Rectangle-BEV Camera Show (Crop).jpg"
    BEV_PathName = pathResultImages + BEV_Name
    print("BEV_PathName:", BEV_PathName)
    imgBEV = cv2.imread(BEV_PathName)
    # Rotate BEV 180 Degree
    #imgBEV = cv2.rotate(imgBEV, cv2.ROTATE_180)
    # Flip Vertical Horizontal
    #imgBEV = cv2.flip(imgBEV, -1)
    print("==> Size New Rectangle Image")
    heightBEV = imgBEV.shape[0]
    widthBEV = imgBEV.shape[1]
    print("Height: ", heightBEV)
    print("Width: ", widthBEV)
    print()

    # Camera Filename Images
    Camera_A_Name = "Camera_A_1.jpg"
    Camera_B_Name = "Camera_B_1.jpg"
    Camera_C_Name = "Camera_C_1.jpg"
    Camera_D_Name = "Camera_D_1.jpg"
    print("=> Filename: ", Camera_A_Name,Camera_B_Name,Camera_C_Name,Camera_D_Name)
    print()

    # Read Camera & Rotate
    imgCameraA = readCamera(Camera_A_Name)
    imgCameraB = readCamera(Camera_B_Name)
    imgCameraC = readCamera(Camera_C_Name)
    imgCameraD = readCamera(Camera_D_Name)

    while True:
        # Show
        cv2.imshow(Camera_A_Name, imgCameraA)
        cv2.imshow(Camera_B_Name, imgCameraB)
        cv2.imshow(Camera_C_Name, imgCameraC)
        cv2.imshow(Camera_D_Name, imgCameraD)
        cv2.imshow(BEV_Name, imgBEV)
        # Get Coordinate Camera
        getCoordinateCamera(imgCameraA,imgCameraB,imgCameraC,imgCameraD,  circlesConvert)
        getCoordinateBEV(imgBEV, circlesBEV)
        # Click Mouse
        clickMouseCamera(Camera_A_Name,Camera_B_Name,Camera_C_Name,Camera_D_Name, mousePoints)
        clickMouseBEV(BEV_Name, mousePointsBEV)
        if (cv2.waitKey(1) & 0xFF == ord('q')) or (counter == 16 and counterBEV == 12):
            break
        cv2.waitKey(1)
    cv2.destroyAllWindows()
    counter = 0
    counterBEV = 0

    # Print Coordinate BEV and Camera
    print("circlesBEV = ")
    print(circlesBEV)
    print("="*50)
    print("circlesConvert = ")
    print(circlesConvert)
    print("="*50)
    print()
    
    # Coordinate Small Square Inside
    pointLeftTop = circlesBEV[0]
    pointRightTop = circlesBEV[1]
    pointRightDwn = circlesBEV[2]
    pointLeftDwn = circlesBEV[3]
    print("Coordinate Small Square Inside = ", pointLeftTop, pointRightTop, pointLeftDwn, pointRightDwn)
    print("="*50)
    print()
    # Show Line Small Square Inside in BEV
    imgSmallSquare = showLineSmallSquare(pointLeftTop, pointRightTop, pointLeftDwn, pointRightDwn, BEV_PathName)
    # Write Image
    cv2.imwrite(pathResultImages + "2.SmallSquare-" + BEV_Name, imgSmallSquare)

    # Matrix Camera -> BEV
    MatrixA = convertBEV(circlesConvert[8],circlesConvert[9],circlesConvert[10],circlesConvert[11], circlesBEV[3],circlesBEV[2],circlesBEV[8],circlesBEV[9])
    MatrixC = convertBEV(circlesConvert[4],circlesConvert[5],circlesConvert[6],circlesConvert[7], circlesBEV[1],circlesBEV[2],circlesBEV[6],circlesBEV[7])
    MatrixD = convertBEV(circlesConvert[0],circlesConvert[1],circlesConvert[2],circlesConvert[3], circlesBEV[0],circlesBEV[1],circlesBEV[4],circlesBEV[5])
    MatrixB = convertBEV(circlesConvert[12],circlesConvert[13],circlesConvert[14],circlesConvert[15], circlesBEV[0],circlesBEV[3],circlesBEV[10],circlesBEV[11])
    
    # Read Camera & Rotate
    imgCameraA = readCamera(Camera_A_Name)
    imgCameraB = readCamera(Camera_B_Name)
    imgCameraC = readCamera(Camera_C_Name)
    imgCameraD = readCamera(Camera_D_Name)

    # Show  BEV Result
    imgCameraA = showBEV_Result(imgCameraA,Camera_A_Name,MatrixA, widthBEV,heightBEV)
    imgCameraB = showBEV_Result(imgCameraB,Camera_B_Name,MatrixB, widthBEV,heightBEV)
    imgCameraC = showBEV_Result(imgCameraC,Camera_C_Name,MatrixC, widthBEV,heightBEV)
    imgCameraD = showBEV_Result(imgCameraD,Camera_D_Name,MatrixD, widthBEV,heightBEV)

    # Write New Image BEV Result
    cv2.imwrite(pathResultImages + "2.BEV-" + Camera_A_Name, imgCameraA)
    cv2.imwrite(pathResultImages + "2.BEV-" + Camera_B_Name, imgCameraB)
    cv2.imwrite(pathResultImages + "2.BEV-" + Camera_C_Name, imgCameraC)
    cv2.imwrite(pathResultImages + "2.BEV-" + Camera_D_Name, imgCameraD)

    # Print Important Parameter
    print()
    print("===> Important  Parameter")
    print("MatrixA = ", MatrixA)
    print("MatrixB = ", MatrixB)
    print("MatrixC = ", MatrixC)
    print("MatrixD = ", MatrixD)

if __name__ == '__main__':       
    # Calling main() function 
    main()


BEV_PathName: Result_Images\\1.Rectangle-BEV Camera Show (Crop).jpg
==> Size New Rectangle Image
Height:  785
Width:  657

=> Filename:  Camera_A_1.jpg Camera_B_1.jpg Camera_C_1.jpg Camera_D_1.jpg

circlesBEV = 
[[224 290]
 [411 298]
 [401 518]
 [230 513]
 [108 203]
 [524 223]
 [518 151]
 [458 650]
 [100 581]
 [528 625]
 [144 151]
 [121 661]]
circlesConvert = 
[[ 98 340]
 [562 351]
 [ 52 244]
 [599 248]
 [118  42]
 [ 86 548]
 [230  43]
 [177 594]
 [ 99 100]
 [576  65]
 [ 26 206]
 [596 200]
 [357  44]
 [379 576]
 [249  24]
 [258 580]]

Coordinate Small Square Inside =  [224 290] [411 298] [230 513] [401 518]


===> Important  Parameter
MatrixA =  [[ 2.36588339e-01 -9.79616203e-01  2.31036674e+02]
 [-9.26623812e-03 -1.35976221e+00  4.85950598e+02]
 [-6.85560659e-05 -3.12789910e-03  1.00000000e+00]]
MatrixB =  [[-2.19189956e+00  1.08059785e-01  4.30714845e+02]
 [-2.98505403e+00 -4.48153295e-01  6.36091901e+02]
 [-7.19621378e-03  4.49285188e-04  1.00000000e+00]]
MatrixC =  [[-1.06122502e+0

[
	{
		"metadata": {
			"outputType": "stream"
		},
		"outputItems": [
			{
				"mimeType": "application/vnd.code.notebook.stdout",
				"data": "BEV_PathName: Result_Images\\\\1.Rectangle-BEV Camera Show (Crop).jpg\n==> Size New Rectangle Image\nHeight:  785\nWidth:  657\n\n=> Filename:  Camera_A_1.jpg Camera_B_1.jpg Camera_C_1.jpg Camera_D_1.jpg\n\ncirclesBEV = \n[[225 290]\n [411 299]\n [399 519]\n [229 512]\n [108 201]\n [523 222]\n [516 150]\n [457 648]\n [100 583]\n [528 624]\n [145 150]\n [121 660]]\n==================================================\ncirclesConvert = \n[[ 99 340]\n [563 348]\n [ 52 244]\n [600 247]\n [116  41]\n [ 88 549]\n [232  42]\n [177 595]\n [ 97  98]\n [577  65]\n [ 27 206]\n [597 198]\n [357  44]\n [382 573]\n [248  23]\n [255 578]]\n==================================================\n\nCoordinate Small Square Inside =  [225 290] [411 299] [229 512] [399 519]\n==================================================\n\n\n===> Important  Parameter\nMatrixA =  [[-6.34108289e-01 -2.27734635e+00  5.04848323e+02]\n [-3.93507530e-02 -2.81662570e+00  5.33345860e+02]\n [-7.30180515e-05 -7.26274092e-03  1.00000000e+00]]\nMatrixB =  [[-2.18932877e+00  9.07495159e-02  4.28473991e+02]\n [-2.96268758e+00 -5.13353589e-01  6.30285749e+02]\n [-7.18884907e-03  3.35444431e-04  1.00000000e+00]]\nMatrixC =  [[-1.01496770e+00 -9.04978235e-02  3.81983601e+02]\n [-1.27786330e+00  1.78625127e-01  3.30447549e+02]\n [-3.07862874e-03 -2.18762067e-04  1.00000000e+00]]\nMatrixD =  [[ 2.43970237e-01 -9.73747510e-01  2.28449199e+02]\n [ 4.20453054e-03 -1.38163826e+00  4.85314862e+02]\n [-4.64693193e-05 -3.17621961e-03  1.00000000e+00]]\n"
			}
		]
	}
]

"2.1 Find Best Small Square Inside - BEV"
==========================================

In [5]:
import cv2
import numpy as np
import glob
import os
import math

pointLeftTop = np.zeros((2), np.int)
pointRightTop = np.zeros((2), np.int)
pointLeftDwn = np.zeros((2), np.int)
pointRightDwn = np.zeros((2), np.int)
pathResultImages = r"Result_Images\\"

# Coordinate Small Square Inside =  [225 290] [411 299] [229 512] [399 519]
pointLeftTop = [225, 290]
pointRightTop = [411, 299]
pointLeftDwn = [229, 512]
pointRightDwn = [399, 519]

def getX_Y_MinMax(leftTop, rghtTop, leftDwn, rghtDwn):
    # == xMin (leftTop & leftDwn)
    if leftTop[0] <= leftDwn[0]:
        xMin = leftTop[0]
    else:
        xMin = leftDwn[0]
    # == xMax (rghtTop & rghtDwn)
    if rghtTop[0] >= rghtDwn[0]:
        xMax = rghtTop[0]
    else:
        xMax = rghtDwn[0]
    # >> yMin (leftTop & rghtTop)
    if leftTop[1] <= rghtTop[1]:
        yMin = leftTop[1]
    else:
        yMin = rghtTop[1]
    # >> yMax (leftDwn & rghtDwn)
    if leftDwn[1] >= rghtDwn[1]:
        yMax = leftDwn[1]
    else:
        yMax = rghtDwn[1]
    return xMin,xMax,yMin,yMax

def showLineSmallSquare(img, xMin,xMax,yMin,yMax):
    color = (0,255,0)
    thickness = 2
    img = cv2.line(img, (xMin,yMin), (xMax,yMin), color, thickness)
    img = cv2.line(img, (xMax,yMin), (xMax,yMax), color, thickness)
    img = cv2.line(img, (xMax,yMax), (xMin,yMax), color, thickness)
    img = cv2.line(img, (xMin,yMax), (xMin,yMin), color, thickness)
    cv2.imshow("showLineSmallSquare", img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    return img


def main():
        #================================================
    # Bird's Eye View Filename Image
    BEV_Name = "1.Rectangle-BEV Camera Show (Crop).jpg"
    BEV_PathName = pathResultImages + BEV_Name
    print("BEV_PathName:", BEV_PathName)
    imgBEV = cv2.imread(BEV_PathName)
    
    print("Coordinate Small Square Inside = ", pointLeftTop, pointRightTop, pointLeftDwn, pointRightDwn)
    # Find X-Y Min & Max Small Square Inside
    pointXMin,pointXMax,pointYMin,pointYMax = getX_Y_MinMax(pointLeftTop, pointRightTop, pointLeftDwn, pointRightDwn)
    print("="*50)
    print("Coordinate Small Square [BEST] Inside = ", pointXMin,pointXMax,pointYMin,pointYMax)

    # Show Line Small Square Inside New
    imgBEVSmallSquare = showLineSmallSquare(imgBEV, pointXMin,pointXMax,pointYMin,pointYMax)
    # Write BEV Small Square
    cv2.imwrite(pathResultImages + "2.SmallSquare-Best-" + BEV_Name, imgBEVSmallSquare)

if __name__ == '__main__':       
    # Calling main() function 
    main()


BEV_PathName: Result_Images\\1.Rectangle-BEV Camera Show (Crop).jpg
Coordinate Small Square Inside =  [225, 290] [411, 299] [229, 512] [399, 519]
Coordinate Small Square [BEST] Inside =  225 411 290 519


BEV_PathName: Result_Images\\1.Rectangle-BEV Camera Show (Crop).jpg
Coordinate Small Square Inside =  [225, 290] [411, 299] [229, 512] [399, 519]
==================================================
Coordinate Small Square [BEST] Inside =  225 411 290 519

"3. Remove Unimportant Parts of Images"
=========================================

In [7]:
import cv2
import numpy as np
import glob
import os
import math

pathResultImages = r"Result_Images\\"

# Coordinate Small Square [BEST] Inside =  225 411 290 519
pointXMin = 225
pointXMax = 411
pointYMin = 290
pointYMax = 519

#>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
#===================================================================================
# removeSmallSquare
def removeSmallSquare(cameraName, imgBEVCamera, xMin,xMax,yMin,yMax):
    thickness = -1
    color = (0,0,0)
    img = cv2.rectangle(imgBEVCamera, (xMin,yMin), (xMax,yMax), color, thickness)
    cv2.imshow("removeSmallSquare", img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    return img
#===================================================================================
# removeSmallSquare [ F U L L ]
def removeSmallSquareFull(cameraName, imgBEVCamera, xMin,xMax,yMin,yMax):
    thickness = -1
    color = (0,0,0)
    if cameraName == "D":
        img = cv2.rectangle(imgBEVCamera, (0,yMin), (imgBEVCamera.shape[1],yMax), color, thickness)
    elif cameraName == "B":
        img = cv2.rectangle(imgBEVCamera, (xMin,0), (xMax,imgBEVCamera.shape[0]), color, thickness)
    elif cameraName == "C":
        img = cv2.rectangle(imgBEVCamera, (xMin,imgBEVCamera.shape[0]), (xMax,0), color, thickness)
    elif cameraName == "A":
        img = cv2.rectangle(imgBEVCamera, (imgBEVCamera.shape[1],yMin), (0,yMax), color, thickness)
    cv2.imshow("removeSmallSquareFull", img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    return img
#>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>

#===================================================================================
def showLineHorizontal(imgBEVCamera,BEV_Camera_Name, halfDistance, legthHorizontal):
    color = (0,0,255)
    thickness = 1
    imgBEVCamera = cv2.line(imgBEVCamera, (0,halfDistance), (legthHorizontal,halfDistance), color, thickness)
    cv2.imshow("Line Remove" + BEV_Camera_Name, imgBEVCamera)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    # Removes Parts
    if BEV_Camera_Name[:14] == "2.BEV-Camera_D":
        # Removes Bottom Parts
        imgBEVCamera = cv2.rectangle(imgBEVCamera, (0,halfDistance), (legthHorizontal,halfDistance*2), (0,0,0), -1)
    elif BEV_Camera_Name[:14] == "2.BEV-Camera_A":
        # Removes Top Parts
        imgBEVCamera = cv2.rectangle(imgBEVCamera, (0,0), (legthHorizontal,halfDistance), (0,0,0), -1)
    cv2.imshow("Remove Parts" + BEV_Camera_Name, imgBEVCamera)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    return imgBEVCamera
def showLineVertical(imgBEVCamera,BEV_Camera_Name, halfDistance, legthVertical):
    color = (0,0,255)
    thickness = 1
    imgBEVCamera = cv2.line(imgBEVCamera, (halfDistance,0), (halfDistance,legthVertical), color, thickness)
    cv2.imshow("Line Remove" + BEV_Camera_Name, imgBEVCamera)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    # Removes Parts
    if BEV_Camera_Name[:14] == "2.BEV-Camera_C":
        # Removes Left Parts
        imgBEVCamera = cv2.rectangle(imgBEVCamera, (0,0), (halfDistance,legthVertical), (0,0,0), -1)
    elif BEV_Camera_Name[:14] == "2.BEV-Camera_B":
        # Removes Right Parts
        imgBEVCamera = cv2.rectangle(imgBEVCamera, (halfDistance,0), (halfDistance*2,legthVertical), (0,0,0), -1)
    cv2.imshow("Remove Parts" + BEV_Camera_Name, imgBEVCamera)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    return imgBEVCamera
#===================================================================================

def main():
    #================================================
    # Bird's Eye View Filename Image
    BEV_Name = "1.Rectangle-BEV Camera Show (Crop).jpg"
    BEV_PathName = pathResultImages + BEV_Name
    print("BEV_PathName:", BEV_PathName)
    imgBEV = cv2.imread(BEV_PathName)
    print("==> Size New Rectangle Image")
    heightBEV = imgBEV.shape[0]
    widthBEV = imgBEV.shape[1]
    print("Height: ", heightBEV)
    print("Width: ", widthBEV)
    print()

    # Bird's Eye View Camera Filename
    BEV_Camera_A_Name = "2.BEV-Camera_A_1.jpg"
    BEV_Camera_B_Name = "2.BEV-Camera_B_1.jpg"
    BEV_Camera_C_Name = "2.BEV-Camera_C_1.jpg"
    BEV_Camera_D_Name = "2.BEV-Camera_D_1.jpg"
    print("=> Filename: ", BEV_Camera_A_Name,BEV_Camera_B_Name,BEV_Camera_C_Name,BEV_Camera_D_Name)
    print()

    # Read Camera [Only]
    print("Read Camera [Only]")
    print()
    imgBEVCameraA = cv2.imread(pathResultImages + BEV_Camera_A_Name)
    imgBEVCameraB = cv2.imread(pathResultImages + BEV_Camera_B_Name)
    imgBEVCameraC = cv2.imread(pathResultImages + BEV_Camera_C_Name)
    imgBEVCameraD = cv2.imread(pathResultImages + BEV_Camera_D_Name)

    print("Remove Small Square [BEST] Inside")
    print()
    #=================================================================
    # Remove Small Square [BEST] Inside
    imgBEVCameraAtest = removeSmallSquare("A", imgBEVCameraA, pointXMin,pointXMax,pointYMin,pointYMax)
    imgBEVCameraBtest = removeSmallSquare("B", imgBEVCameraB, pointXMin,pointXMax,pointYMin,pointYMax)
    imgBEVCameraCtest = removeSmallSquare("C", imgBEVCameraC, pointXMin,pointXMax,pointYMin,pointYMax)
    imgBEVCameraDtest = removeSmallSquare("D", imgBEVCameraD, pointXMin,pointXMax,pointYMin,pointYMax)
    '''
    #=================================================================
    # Remove Small Square [BEST] Inside [ F U L L ]
    imgBEVCameraFullA = removeSmallSquareFull("A", imgBEVCameraA, pointXMin,pointXMax,pointYMin,pointYMax)
    imgBEVCameraFullB = removeSmallSquareFull("B", imgBEVCameraB, pointXMin,pointXMax,pointYMin,pointYMax)
    imgBEVCameraFullC = removeSmallSquareFull("C", imgBEVCameraC, pointXMin,pointXMax,pointYMin,pointYMax)
    imgBEVCameraFullD = removeSmallSquareFull("D", imgBEVCameraD, pointXMin,pointXMax,pointYMin,pointYMax)
    #=================================================================
    '''

    print("Show Line to Divided and Removes")
    print()
    # Divided Into 2 Parts
    halfHeight = int(heightBEV/2)
    halfWidth = int(widthBEV/2)
    #=================================================================
    # Show Line to Divided and Removes
    imgBEVCameraRemoveA = showLineHorizontal(imgBEVCameraAtest,BEV_Camera_A_Name, halfHeight,widthBEV)
    imgBEVCameraRemoveB = showLineVertical(imgBEVCameraBtest,BEV_Camera_B_Name, halfWidth,heightBEV)
    imgBEVCameraRemoveC = showLineVertical(imgBEVCameraCtest,BEV_Camera_C_Name, halfWidth,heightBEV)
    imgBEVCameraRemoveD = showLineHorizontal(imgBEVCameraDtest,BEV_Camera_D_Name, halfHeight,widthBEV)
    '''
    #=================================================================
    # Show Line to Divided and Removes [ F U L L ]
    imgBEVCameraRemoveFullA = showLineHorizontal(imgBEVCameraFullA,BEV_Camera_A_Name, halfHeight,widthBEV)
    imgBEVCameraRemoveFullB = showLineVertical(imgBEVCameraFullB,BEV_Camera_B_Name, halfWidth,heightBEV)
    imgBEVCameraRemoveFullC = showLineVertical(imgBEVCameraFullC,BEV_Camera_C_Name, halfWidth,heightBEV)
    imgBEVCameraRemoveFullD = showLineHorizontal(imgBEVCameraFullD,BEV_Camera_D_Name, halfHeight,widthBEV)
    #=================================================================
    '''

    print("Write new Images")
    print()
    #=================================================================  
    # Write new Images
    cv2.imwrite(pathResultImages + "3.BEV-Remove-" + BEV_Camera_A_Name[6:], imgBEVCameraRemoveA)
    cv2.imwrite(pathResultImages + "3.BEV-Remove-" + BEV_Camera_B_Name[6:], imgBEVCameraRemoveB)
    cv2.imwrite(pathResultImages + "3.BEV-Remove-" + BEV_Camera_C_Name[6:], imgBEVCameraRemoveC)
    cv2.imwrite(pathResultImages + "3.BEV-Remove-" + BEV_Camera_D_Name[6:], imgBEVCameraRemoveD)
    '''
    #=================================================================
    # Write new Images [ F U L L ]
    cv2.imwrite(pathResultImages + "3.BEV-Remove-Full-" + BEV_Camera_A_Name[6:], imgBEVCameraRemoveFullA)
    cv2.imwrite(pathResultImages + "3.BEV-Remove-Full-" + BEV_Camera_B_Name[6:], imgBEVCameraRemoveFullB)
    cv2.imwrite(pathResultImages + "3.BEV-Remove-Full-" + BEV_Camera_C_Name[6:], imgBEVCameraRemoveFullC)
    cv2.imwrite(pathResultImages + "3.BEV-Remove-Full-" + BEV_Camera_D_Name[6:], imgBEVCameraRemoveFullD)
    #=================================================================
    '''

if __name__ == '__main__':       
    # Calling main() function 
    main()

BEV_PathName: Result_Images\\1.Rectangle-BEV Camera Show (Crop).jpg
==> Size New Rectangle Image
Height:  785
Width:  657

=> Filename:  2.BEV-Camera_A_1.jpg 2.BEV-Camera_B_1.jpg 2.BEV-Camera_C_1.jpg 2.BEV-Camera_D_1.jpg

Read Camera [Only]

Remove Small Square [BEST] Inside

Show Line to Divided and Removes

Write new Images



"3.1 Remove Unimportant Parts of Images [ F U L L ]"
=========================================

In [8]:
import cv2
import numpy as np
import glob
import os
import math

pathResultImages = r"Result_Images\\"

# Coordinate Small Square [BEST] Inside =  225 411 290 519
pointXMin = 225
pointXMax = 411
pointYMin = 290
pointYMax = 519

#>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
#===================================================================================
# removeSmallSquare
def removeSmallSquare(cameraName, imgBEVCamera, xMin,xMax,yMin,yMax):
    thickness = -1
    color = (0,0,0)
    img = cv2.rectangle(imgBEVCamera, (xMin,yMin), (xMax,yMax), color, thickness)
    cv2.imshow("removeSmallSquare", img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    return img
#===================================================================================
# removeSmallSquare [ F U L L ]
def removeSmallSquareFull(cameraName, imgBEVCamera, xMin,xMax,yMin,yMax):
    thickness = -1
    color = (0,0,0)
    if cameraName == "D":
        img = cv2.rectangle(imgBEVCamera, (0,yMin), (imgBEVCamera.shape[1],yMax), color, thickness)
    elif cameraName == "B":
        img = cv2.rectangle(imgBEVCamera, (xMin,0), (xMax,imgBEVCamera.shape[0]), color, thickness)
    elif cameraName == "C":
        img = cv2.rectangle(imgBEVCamera, (xMin,imgBEVCamera.shape[0]), (xMax,0), color, thickness)
    elif cameraName == "A":
        img = cv2.rectangle(imgBEVCamera, (imgBEVCamera.shape[1],yMin), (0,yMax), color, thickness)
    cv2.imshow("removeSmallSquareFull", img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    return img
#>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>

#===================================================================================
def showLineHorizontal(imgBEVCamera,BEV_Camera_Name, halfDistance, legthHorizontal):
    color = (0,0,255)
    thickness = 1
    imgBEVCamera = cv2.line(imgBEVCamera, (0,halfDistance), (legthHorizontal,halfDistance), color, thickness)
    cv2.imshow("Line Remove" + BEV_Camera_Name, imgBEVCamera)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    # Removes Parts
    if BEV_Camera_Name[:14] == "2.BEV-Camera_D":
        # Removes Buttom Parts
        imgBEVCamera = cv2.rectangle(imgBEVCamera, (0,halfDistance), (legthHorizontal,halfDistance*2), (0,0,0), -1)
    elif BEV_Camera_Name[:14] == "2.BEV-Camera_A":
        # Removes Top Parts
        imgBEVCamera = cv2.rectangle(imgBEVCamera, (0,0), (legthHorizontal,halfDistance), (0,0,0), -1)
    cv2.imshow("Remove Parts" + BEV_Camera_Name, imgBEVCamera)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    return imgBEVCamera
def showLineVertical(imgBEVCamera,BEV_Camera_Name, halfDistance, legthVertical):
    color = (0,0,255)
    thickness = 1
    imgBEVCamera = cv2.line(imgBEVCamera, (halfDistance,0), (halfDistance,legthVertical), color, thickness)
    cv2.imshow("Line Remove" + BEV_Camera_Name, imgBEVCamera)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    # Removes Parts
    if BEV_Camera_Name[:14] == "2.BEV-Camera_C":
        # Removes Left Parts
        imgBEVCamera = cv2.rectangle(imgBEVCamera, (0,0), (halfDistance,legthVertical), (0,0,0), -1)
    elif BEV_Camera_Name[:14] == "2.BEV-Camera_B":
        # Removes Right Parts
        imgBEVCamera = cv2.rectangle(imgBEVCamera, (halfDistance,0), (halfDistance*2,legthVertical), (0,0,0), -1)
    cv2.imshow("Remove Parts" + BEV_Camera_Name, imgBEVCamera)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    return imgBEVCamera
#===================================================================================

def main():
    #================================================
    # Bird's Eye View Filename Image
    BEV_Name = "1.Rectangle-BEV Camera Show (Crop).jpg"
    BEV_PathName = pathResultImages + BEV_Name
    print("BEV_PathName:", BEV_PathName)
    imgBEV = cv2.imread(BEV_PathName)
    print("==> Size New Rectangle Image")
    heightBEV = imgBEV.shape[0]
    widthBEV = imgBEV.shape[1]
    print("Height: ", heightBEV)
    print("Width: ", widthBEV)
    print()

    # Bird's Eye View Camera Filename
    BEV_Camera_A_Name = "2.BEV-Camera_A_1.jpg"
    BEV_Camera_B_Name = "2.BEV-Camera_B_1.jpg"
    BEV_Camera_C_Name = "2.BEV-Camera_C_1.jpg"
    BEV_Camera_D_Name = "2.BEV-Camera_D_1.jpg"
    print("=> Filename: ", BEV_Camera_A_Name,BEV_Camera_B_Name,BEV_Camera_C_Name,BEV_Camera_D_Name)
    print()

    # Read Camera [Only]
    print("Read Camera [Only]")
    print()
    imgBEVCameraA = cv2.imread(pathResultImages + BEV_Camera_A_Name)
    imgBEVCameraB = cv2.imread(pathResultImages + BEV_Camera_B_Name)
    imgBEVCameraC = cv2.imread(pathResultImages + BEV_Camera_C_Name)
    imgBEVCameraD = cv2.imread(pathResultImages + BEV_Camera_D_Name)

    print("Remove Small Square [BEST] Inside")
    print()
    '''
    #=================================================================
    # Remove Small Square [BEST] Inside
    imgBEVCameraAtest = removeSmallSquare("A", imgBEVCameraA, pointXMin,pointXMax,pointYMin,pointYMax)
    imgBEVCameraBtest = removeSmallSquare("B", imgBEVCameraB, pointXMin,pointXMax,pointYMin,pointYMax)
    imgBEVCameraCtest = removeSmallSquare("C", imgBEVCameraC, pointXMin,pointXMax,pointYMin,pointYMax)
    imgBEVCameraDtest = removeSmallSquare("D", imgBEVCameraD, pointXMin,pointXMax,pointYMin,pointYMax)
    '''
    #=================================================================
    # Remove Small Square [BEST] Inside [ F U L L ]
    imgBEVCameraFullA = removeSmallSquareFull("A", imgBEVCameraA, pointXMin,pointXMax,pointYMin,pointYMax)
    imgBEVCameraFullB = removeSmallSquareFull("B", imgBEVCameraB, pointXMin,pointXMax,pointYMin,pointYMax)
    imgBEVCameraFullC = removeSmallSquareFull("C", imgBEVCameraC, pointXMin,pointXMax,pointYMin,pointYMax)
    imgBEVCameraFullD = removeSmallSquareFull("D", imgBEVCameraD, pointXMin,pointXMax,pointYMin,pointYMax)
    #=================================================================
    

    print("Show Line to Divided and Removes")
    print()
    # Divided Into 2 Parts
    halfHeight = int(heightBEV/2)
    halfWidth = int(widthBEV/2)
    '''
    #=================================================================
    # Show Line to Divided and Removes
    imgBEVCameraRemoveA = showLineHorizontal(imgBEVCameraAtest,BEV_Camera_A_Name, halfHeight,widthBEV)
    imgBEVCameraRemoveB = showLineVertical(imgBEVCameraBtest,BEV_Camera_B_Name, halfWidth,heightBEV)
    imgBEVCameraRemoveC = showLineVertical(imgBEVCameraCtest,BEV_Camera_C_Name, halfWidth,heightBEV)
    imgBEVCameraRemoveD = showLineHorizontal(imgBEVCameraDtest,BEV_Camera_D_Name, halfHeight,widthBEV)
    '''
    #=================================================================
    # Show Line to Divided and Removes [ F U L L ]
    imgBEVCameraRemoveFullA = showLineHorizontal(imgBEVCameraFullA,BEV_Camera_A_Name, halfHeight,widthBEV)
    imgBEVCameraRemoveFullB = showLineVertical(imgBEVCameraFullB,BEV_Camera_B_Name, halfWidth,heightBEV)
    imgBEVCameraRemoveFullC = showLineVertical(imgBEVCameraFullC,BEV_Camera_C_Name, halfWidth,heightBEV)
    imgBEVCameraRemoveFullD = showLineHorizontal(imgBEVCameraFullD,BEV_Camera_D_Name, halfHeight,widthBEV)
    #=================================================================
    

    print("Write new Images")
    print()
    '''
    #=================================================================  
    # Write new Images
    cv2.imwrite(pathResultImages + "3.BEV-Remove-" + BEV_Camera_A_Name[6:], imgBEVCameraRemoveA)
    cv2.imwrite(pathResultImages + "3.BEV-Remove-" + BEV_Camera_B_Name[6:], imgBEVCameraRemoveB)
    cv2.imwrite(pathResultImages + "3.BEV-Remove-" + BEV_Camera_C_Name[6:], imgBEVCameraRemoveC)
    cv2.imwrite(pathResultImages + "3.BEV-Remove-" + BEV_Camera_D_Name[6:], imgBEVCameraRemoveD)
    '''
    #=================================================================
    # Write new Images [ F U L L ]
    cv2.imwrite(pathResultImages + "3.BEV-Remove-Full-" + BEV_Camera_A_Name[6:], imgBEVCameraRemoveFullA)
    cv2.imwrite(pathResultImages + "3.BEV-Remove-Full-" + BEV_Camera_B_Name[6:], imgBEVCameraRemoveFullB)
    cv2.imwrite(pathResultImages + "3.BEV-Remove-Full-" + BEV_Camera_C_Name[6:], imgBEVCameraRemoveFullC)
    cv2.imwrite(pathResultImages + "3.BEV-Remove-Full-" + BEV_Camera_D_Name[6:], imgBEVCameraRemoveFullD)
    #=================================================================

if __name__ == '__main__':       
    # Calling main() function 
    main()

BEV_PathName: Result_Images\\1.Rectangle-BEV Camera Show (Crop).jpg
==> Size New Rectangle Image
Height:  785
Width:  657

=> Filename:  2.BEV-Camera_A_1.jpg 2.BEV-Camera_B_1.jpg 2.BEV-Camera_C_1.jpg 2.BEV-Camera_D_1.jpg

Read Camera [Only]

Remove Small Square [BEST] Inside

Show Line to Divided and Removes

Write new Images



"4. Crop Half Parts"
=======================================

In [9]:
import cv2
import numpy as np
import glob
import os
import math

pathResultImages = r"Result_Images\\"

def cropBEVRemovaCamera(cameraName,pathImages,BEVRemove_Camera_Name):
    img = cv2.imread(pathImages + BEVRemove_Camera_Name)
    height = img.shape[0]
    width = img.shape[1]
    if cameraName == "D":
        x = 0
        h = int(height/2)
        y = 0
        w = width
        img = img[y:y+h, x:x+w]
        imgRotate = img
    elif cameraName == "B":
        x = 0
        h = height
        y = 0
        w = int(width/2)
        img = img[y:y+h, x:x+w]
        # Rotate 
        imgRotate = cv2.rotate(img, cv2.ROTATE_90_CLOCKWISE)
    elif cameraName == "C":
        x = int(width/2)
        h = height
        y = 0
        w = int(width/2)
        img = img[y:y+h, x:x+w]
        imgRotate = cv2.rotate(img, cv2.ROTATE_90_COUNTERCLOCKWISE)
    elif cameraName == "A":
        x = 0
        h = int(height/2)
        y = int(height/2)
        w = width
        img = img[y:y+h, x:x+w]
        imgRotate = cv2.rotate(img, cv2.ROTATE_180)
    cv2.imshow("-Crop-" + BEVRemove_Camera_Name, img)
    cv2.imshow("-Rotate-" + BEVRemove_Camera_Name, imgRotate)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    return img, imgRotate


def main():
    # Bird's Eye View - Remove - Camera Filename
    BEVRemove_Camera_A_Name = "3.BEV-Remove-Camera_A_1.jpg"
    BEVRemove_Camera_B_Name = "3.BEV-Remove-Camera_B_1.jpg"
    BEVRemove_Camera_C_Name = "3.BEV-Remove-Camera_C_1.jpg"
    BEVRemove_Camera_D_Name = "3.BEV-Remove-Camera_D_1.jpg"
    print("=> Filename: ", BEVRemove_Camera_A_Name,BEVRemove_Camera_B_Name,BEVRemove_Camera_C_Name,BEVRemove_Camera_D_Name)
    print()

    # Read BEV Remove Camera & Crop
    imgBEVRemoveCropCameraA, imgBEVRemoveRotateCameraA = cropBEVRemovaCamera("A", pathResultImages,BEVRemove_Camera_A_Name)
    imgBEVRemoveCropCameraB, imgBEVRemoveRotateCameraB = cropBEVRemovaCamera("B", pathResultImages,BEVRemove_Camera_B_Name)
    imgBEVRemoveCropCameraC, imgBEVRemoveRotateCameraC = cropBEVRemovaCamera("C", pathResultImages,BEVRemove_Camera_C_Name)
    imgBEVRemoveCropCameraD, imgBEVRemoveRotateCameraD = cropBEVRemovaCamera("D", pathResultImages,BEVRemove_Camera_D_Name)

    # Write new Images
    cv2.imwrite(pathResultImages + "4.1.BEV-Remove-Crop-" + BEVRemove_Camera_A_Name[13:], imgBEVRemoveCropCameraA)
    cv2.imwrite(pathResultImages + "4.1.BEV-Remove-Crop-" + BEVRemove_Camera_B_Name[13:], imgBEVRemoveCropCameraB)
    cv2.imwrite(pathResultImages + "4.1.BEV-Remove-Crop-" + BEVRemove_Camera_C_Name[13:], imgBEVRemoveCropCameraC)
    cv2.imwrite(pathResultImages + "4.1.BEV-Remove-Crop-" + BEVRemove_Camera_D_Name[13:], imgBEVRemoveCropCameraD)

    # Write new Images Rotate
    cv2.imwrite(pathResultImages + "4.2.BEV-Remove-Rotate-" + BEVRemove_Camera_A_Name[13:], imgBEVRemoveRotateCameraA)
    cv2.imwrite(pathResultImages + "4.2.BEV-Remove-Rotate-" + BEVRemove_Camera_B_Name[13:], imgBEVRemoveRotateCameraB)
    cv2.imwrite(pathResultImages + "4.2.BEV-Remove-Rotate-" + BEVRemove_Camera_C_Name[13:], imgBEVRemoveRotateCameraC)
    cv2.imwrite(pathResultImages + "4.2.BEV-Remove-Rotate-" + BEVRemove_Camera_D_Name[13:], imgBEVRemoveRotateCameraD)


if __name__ == '__main__':       
    # Calling main() function 
    main()

=> Filename:  3.BEV-Remove-Camera_A_1.jpg 3.BEV-Remove-Camera_B_1.jpg 3.BEV-Remove-Camera_C_1.jpg 3.BEV-Remove-Camera_D_1.jpg



"4.1 Crop Half Parts [ F U L L ]"
=======================================

In [10]:
import cv2
import numpy as np
import glob
import os
import math

pathResultImages = r"Result_Images\\"

def cropBEVRemovaCamera(cameraName,pathImages,BEVRemove_Camera_Name):
    img = cv2.imread(pathImages + BEVRemove_Camera_Name)
    height = img.shape[0]
    width = img.shape[1]
    if cameraName == "D":
        x = 0
        h = int(height/2)
        y = 0
        w = width
        img = img[y:y+h, x:x+w]
        imgRotate = img
    elif cameraName == "B":
        x = 0
        h = height
        y = 0
        w = int(width/2)
        img = img[y:y+h, x:x+w]
        # Rotate 
        imgRotate = cv2.rotate(img, cv2.ROTATE_90_CLOCKWISE)
    elif cameraName == "C":
        x = int(width/2)
        h = height
        y = 0
        w = int(width/2)
        img = img[y:y+h, x:x+w]
        imgRotate = cv2.rotate(img, cv2.ROTATE_90_COUNTERCLOCKWISE)
    elif cameraName == "A":
        x = 0
        h = int(height/2)
        y = int(height/2)
        w = width
        img = img[y:y+h, x:x+w]
        imgRotate = cv2.rotate(img, cv2.ROTATE_180)
    cv2.imshow("-Crop-" + BEVRemove_Camera_Name, img)
    cv2.imshow("-Rotate-" + BEVRemove_Camera_Name, imgRotate)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    return img, imgRotate


def main():
    # Bird's Eye View - Remove - Camera Filename [ F U L L ]
    BEVRemove_Camera_A_Name = "3.BEV-Remove-Full-Camera_A_1.jpg"
    BEVRemove_Camera_B_Name = "3.BEV-Remove-Full-Camera_B_1.jpg"
    BEVRemove_Camera_C_Name = "3.BEV-Remove-Full-Camera_C_1.jpg"
    BEVRemove_Camera_D_Name = "3.BEV-Remove-Full-Camera_D_1.jpg"
    print("=> Filename: ", BEVRemove_Camera_A_Name,BEVRemove_Camera_B_Name,BEVRemove_Camera_C_Name,BEVRemove_Camera_D_Name)
    print()

    # Read BEV Remove Camera & Crop
    imgBEVRemoveCropCameraA, imgBEVRemoveRotateCameraA = cropBEVRemovaCamera("A", pathResultImages,BEVRemove_Camera_A_Name)
    imgBEVRemoveCropCameraB, imgBEVRemoveRotateCameraB = cropBEVRemovaCamera("B", pathResultImages,BEVRemove_Camera_B_Name)
    imgBEVRemoveCropCameraC, imgBEVRemoveRotateCameraC = cropBEVRemovaCamera("C", pathResultImages,BEVRemove_Camera_C_Name)
    imgBEVRemoveCropCameraD, imgBEVRemoveRotateCameraD = cropBEVRemovaCamera("D", pathResultImages,BEVRemove_Camera_D_Name)

    # Write new Images
    cv2.imwrite(pathResultImages + "4.1.BEV-Remove-Crop-Full-" + BEVRemove_Camera_A_Name[13:], imgBEVRemoveCropCameraA)
    cv2.imwrite(pathResultImages + "4.1.BEV-Remove-Crop-Full-" + BEVRemove_Camera_B_Name[13:], imgBEVRemoveCropCameraB)
    cv2.imwrite(pathResultImages + "4.1.BEV-Remove-Crop-Full-" + BEVRemove_Camera_C_Name[13:], imgBEVRemoveCropCameraC)
    cv2.imwrite(pathResultImages + "4.1.BEV-Remove-Crop-Full-" + BEVRemove_Camera_D_Name[13:], imgBEVRemoveCropCameraD)

    # Write new Images Rotate
    cv2.imwrite(pathResultImages + "4.2.BEV-Remove-Rotate-Full-" + BEVRemove_Camera_A_Name[13:], imgBEVRemoveRotateCameraA)
    cv2.imwrite(pathResultImages + "4.2.BEV-Remove-Rotate-Full-" + BEVRemove_Camera_B_Name[13:], imgBEVRemoveRotateCameraB)
    cv2.imwrite(pathResultImages + "4.2.BEV-Remove-Rotate-Full-" + BEVRemove_Camera_C_Name[13:], imgBEVRemoveRotateCameraC)
    cv2.imwrite(pathResultImages + "4.2.BEV-Remove-Rotate-Full-" + BEVRemove_Camera_D_Name[13:], imgBEVRemoveRotateCameraD)


if __name__ == '__main__':       
    # Calling main() function 
    main()

=> Filename:  3.BEV-Remove-Full-Camera_A_1.jpg 3.BEV-Remove-Full-Camera_B_1.jpg 3.BEV-Remove-Full-Camera_C_1.jpg 3.BEV-Remove-Full-Camera_D_1.jpg



" [ *** ============ F I N A L ============ ***] "
==========================

In [11]:
import cv2
import numpy as np
import glob
import os
import math

# =============== [ P A R A M E T E R ]
#=========================================================
pathResultImages = r"Result_Images\\"
BEV_Name = "1.Rectangle-BEV Camera Show (Crop).jpg"
heightBEV = 785
widthBEV = 657
# =============== Camera Filename Images
Camera_A_Name = "Camera_A_1.jpg"
Camera_B_Name = "Camera_B_1.jpg"
Camera_C_Name = "Camera_C_1.jpg"
Camera_D_Name = "Camera_D_1.jpg"
print("=> Filename: ", Camera_A_Name,Camera_B_Name,Camera_C_Name,Camera_D_Name)
print()
# =============== Matrix Camera -> BEV
MatrixA = np.zeros((3,3), np.float)
MatrixB = np.zeros((3,3), np.float)
MatrixC = np.zeros((3,3), np.float)
MatrixD = np.zeros((3,3), np.float)
MatrixA[0:3] = [[ 2.36588339e-01, -9.79616203e-01,  2.31036674e+02], [-9.26623812e-03, -1.35976221e+00,  4.85950598e+02], [-6.85560659e-05, -3.12789910e-03,  1.00000000e+00]]
MatrixB[0:3] = [[-2.19189956e+00,  1.08059785e-01,  4.30714845e+02], [-2.98505403e+00, -4.48153295e-01,  6.36091901e+02], [-7.19621378e-03,  4.49285188e-04,  1.00000000e+00]]
MatrixC[0:3] = [[-1.06122502e+00, -5.80395453e-02,  3.81511526e+02], [-1.29407427e+00,  2.14188277e-01,  3.27761048e+02], [-3.18749314e-03, -1.48513790e-04,  1.00000000e+00]]
MatrixD[0:3] = [[-6.56100043e-01, -2.23582382e+00,  4.97126571e+02], [-4.64843971e-02, -2.77641520e+00,  5.24733577e+02], [-1.21516486e-04, -7.20435663e-03,  1.00000000e+00]]
print("=> MatrixA = ", MatrixA)
print("=> MatrixB = ", MatrixB)
print("=> MatrixC = ", MatrixC)
print("=> MatrixD = ", MatrixD)
print("="*50)
print()
# =============== Coordinate Small Square [BEST] Inside    
pointXMin = 225
pointXMax = 411
pointYMin = 290
pointYMax = 519
print("=> pointXMin = ", pointXMin)
print("=> pointXMax = ", pointXMax)
print("=> pointYMin = ", pointYMin)
print("=> pointYMax = ", pointYMax)
print("="*50)
print()
#=========================================================
# =============== [ P A R A M E T E R ]

# =============== [ F U N C T I O N ]
#=========================================================
def readCamera(cameraName, Camera_Filename):
    img = cv2.imread(Camera_Filename)
    if (cameraName == "D"):
        # Rotate D - No Rotate
        pass
    elif (cameraName == "B"):
        # Camera B - Rotate 90 COUNTERCLOCKWISE
        img = cv2.rotate(img, cv2.cv2.ROTATE_90_COUNTERCLOCKWISE)
    elif (cameraName == "C"):
        # Camera C - Rotate 90 CLOCKWISE
        img = cv2.rotate(img, cv2.cv2.ROTATE_90_CLOCKWISE)
    elif (cameraName == "A"):
        # Camera A - Rotate 180 
        img = cv2.rotate(img, cv2.cv2.ROTATE_180)
    cv2.imshow(Camera_Filename, img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    return img
#=========================================================
#>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
def showBEV_Result(img,Camera_Filename,matrix, width,height):
    imgBEV = cv2.warpPerspective(img,matrix, (width,height))
    '''
    cv2.imshow("-BEV-" + Camera_Filename, imgBEV)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    '''
    return imgBEV
#>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
# removeSmallSquare
def removeSmallSquare(cameraName, imgBEVCamera, xMin,xMax,yMin,yMax):
    thickness = -1
    color = (0,0,0)
    img = cv2.rectangle(imgBEVCamera, (xMin,yMin), (xMax,yMax), color, thickness)
    '''
    cv2.imshow("removeSmallSquare", img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    '''
    return img
#>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
def showLineDividedRemove(cameraName,imgBEVCamera,BEV_Camera_Name, heightBEV,widthBEV):
    color = (0,0,255)
    thickness = 1
    # Divided Into 2 Parts
    halfHeight = int(heightBEV/2)
    halfWidth = int(widthBEV/2)
    if cameraName == "A" or cameraName == "D":
        imgBEVCamera = cv2.line(imgBEVCamera, (0,halfHeight), (widthBEV,halfHeight), color, thickness)
        if cameraName == "D":
            # Removes Buttom Parts
            imgBEVCamera = cv2.rectangle(imgBEVCamera, (0,halfHeight), (widthBEV,halfHeight*2), (0,0,0), -1)
        elif cameraName == "A":
            # Removes Top Parts
            imgBEVCamera = cv2.rectangle(imgBEVCamera, (0,0), (widthBEV,halfHeight), (0,0,0), -1)
    elif cameraName == "C" or cameraName == "B":
        imgBEVCamera = cv2.line(imgBEVCamera, (halfWidth,0), (halfWidth,heightBEV), color, thickness)
        if cameraName == "C":
            # Removes Left Parts
            imgBEVCamera = cv2.rectangle(imgBEVCamera, (0,0), (halfWidth,heightBEV), (0,0,0), -1)
        elif cameraName == "B":
            # Removes Right Parts
            imgBEVCamera = cv2.rectangle(imgBEVCamera, (halfWidth,0), (halfWidth*2,heightBEV), (0,0,0), -1)
    '''
    cv2.imshow("Remove Parts" + BEV_Camera_Name, imgBEVCamera)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    '''
    return imgBEVCamera
#>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
def cropBEVRemovaCamera(cameraName,img,BEVRemove_Camera_Name):
    height = img.shape[0]
    width = img.shape[1]
    if cameraName == "D":
        x = 0
        h = int(height/2)
        y = 0
        w = width
        img = img[y:y+h, x:x+w]
        imgRotate = img
    elif cameraName == "B":
        x = 0
        h = height
        y = 0
        w = int(width/2)
        img = img[y:y+h, x:x+w]
        # Rotate 
        imgRotate = cv2.rotate(img, cv2.ROTATE_90_CLOCKWISE)
    elif cameraName == "C":
        x = int(width/2)
        h = height
        y = 0
        w = int(width/2)
        img = img[y:y+h, x:x+w]
        imgRotate = cv2.rotate(img, cv2.ROTATE_90_COUNTERCLOCKWISE)
    elif cameraName == "A":
        x = 0
        h = int(height/2)
        y = int(height/2)
        w = width
        img = img[y:y+h, x:x+w]
        imgRotate = cv2.rotate(img, cv2.ROTATE_180)
    cv2.imshow("-Crop-" + BEVRemove_Camera_Name, img)
    cv2.imshow("-Rotate-" + BEVRemove_Camera_Name, imgRotate)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    return img, imgRotate
#>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
#=========================================================
def writeNewImage(img, pathWriteImg):
    cv2.imwrite(pathWriteImg, img)
#=========================================================
# =============== [ F U N C T I O N ]

# =============== [ M A I N - F U N C T I O N ]
#=========================================================
def main():
    print("==> 1. Size BEV")
    print("Height: ", heightBEV)
    print("Width: ", widthBEV)
    print()

    print("==> 2. Read Camera & Rotate")
    print()
    # ==================== Read Camera & Rotate
    imgCameraA = readCamera("A", Camera_A_Name)
    imgCameraB = readCamera("B", Camera_B_Name)
    imgCameraC = readCamera("C", Camera_C_Name)
    imgCameraD = readCamera("D", Camera_D_Name)

    print("==> 2. Show  BEV Result")
    print()
    # ==================== Show  BEV Result
    imgCameraA = showBEV_Result(imgCameraA,Camera_A_Name,MatrixA, widthBEV,heightBEV)
    imgCameraB = showBEV_Result(imgCameraB,Camera_B_Name,MatrixB, widthBEV,heightBEV)
    imgCameraC = showBEV_Result(imgCameraC,Camera_C_Name,MatrixC, widthBEV,heightBEV)
    imgCameraD = showBEV_Result(imgCameraD,Camera_D_Name,MatrixD, widthBEV,heightBEV)

    print("==> 3. Remove Small Square [BEST] Inside")
    print()
    # ==================== Remove Small Square [BEST] Inside
    imgCameraA = removeSmallSquare("A", imgCameraA, pointXMin,pointXMax,pointYMin,pointYMax)
    imgCameraB = removeSmallSquare("B", imgCameraB, pointXMin,pointXMax,pointYMin,pointYMax)
    imgCameraC = removeSmallSquare("C", imgCameraC, pointXMin,pointXMax,pointYMin,pointYMax)
    imgCameraD = removeSmallSquare("D", imgCameraD, pointXMin,pointXMax,pointYMin,pointYMax)

    print("==> 3. Show Line to Divided and Removes")
    print()
    # ==================== Show Line to Divided and Removes
    imgCameraA = showLineDividedRemove("A",imgCameraA,Camera_A_Name, heightBEV,widthBEV)
    imgCameraB = showLineDividedRemove("B",imgCameraB,Camera_B_Name, heightBEV,widthBEV)
    imgCameraC = showLineDividedRemove("C",imgCameraC,Camera_C_Name, heightBEV,widthBEV)
    imgCameraD = showLineDividedRemove("D",imgCameraD,Camera_D_Name, heightBEV,widthBEV)

    print("==> 4. Show Line to Divided and Removes")
    print()
    # ==================== Read BEV Remove Camera & Crop
    imgCameraA, imgCameraRotateA = cropBEVRemovaCamera("A", imgCameraA,Camera_A_Name)
    imgCameraB, imgCameraRotateB = cropBEVRemovaCamera("B", imgCameraB,Camera_B_Name)
    imgCameraC, imgCameraRotateC = cropBEVRemovaCamera("C", imgCameraC,Camera_C_Name)
    imgCameraD, imgCameraRotateD = cropBEVRemovaCamera("D", imgCameraD,Camera_D_Name)

    print("==> 4. Write new Images")
    print()
    # ==================== Write new Images
    filenameImage = "5.1.BEV-FINAL-"
    filenameImage = pathResultImages + filenameImage
    writeNewImage(imgCameraA, filenameImage+Camera_A_Name)
    writeNewImage(imgCameraB, filenameImage+Camera_B_Name)
    writeNewImage(imgCameraC, filenameImage+Camera_C_Name)
    writeNewImage(imgCameraD, filenameImage+Camera_D_Name)
    # ==================== Write new Images Rotate
    filenameImage = "5.2.R.BEV-FINAL-"
    filenameImage = pathResultImages + filenameImage
    writeNewImage(imgCameraRotateA, filenameImage+Camera_A_Name)
    writeNewImage(imgCameraRotateB, filenameImage+Camera_B_Name)
    writeNewImage(imgCameraRotateC, filenameImage+Camera_C_Name)
    writeNewImage(imgCameraRotateD, filenameImage+Camera_D_Name)
#=========================================================
# =============== [ M A I N - F U N C T I O N ]

if __name__ == '__main__':       
    # Calling main() function 
    main()

=> Filename:  Camera_A_1.jpg Camera_B_1.jpg Camera_C_1.jpg Camera_D_1.jpg

=> MatrixA =  [[ 2.36588339e-01 -9.79616203e-01  2.31036674e+02]
 [-9.26623812e-03 -1.35976221e+00  4.85950598e+02]
 [-6.85560659e-05 -3.12789910e-03  1.00000000e+00]]
=> MatrixB =  [[-2.19189956e+00  1.08059785e-01  4.30714845e+02]
 [-2.98505403e+00 -4.48153295e-01  6.36091901e+02]
 [-7.19621378e-03  4.49285188e-04  1.00000000e+00]]
=> MatrixC =  [[-1.06122502e+00 -5.80395453e-02  3.81511526e+02]
 [-1.29407427e+00  2.14188277e-01  3.27761048e+02]
 [-3.18749314e-03 -1.48513790e-04  1.00000000e+00]]
=> MatrixD =  [[-6.56100043e-01 -2.23582382e+00  4.97126571e+02]
 [-4.64843971e-02 -2.77641520e+00  5.24733577e+02]
 [-1.21516486e-04 -7.20435663e-03  1.00000000e+00]]

=> pointXMin =  225
=> pointXMax =  411
=> pointYMin =  290
=> pointYMax =  519

==> 1. Size BEV
Height:  785
Width:  657

==> 2. Read Camera & Rotate

==> 2. Show  BEV Result

==> 3. Remove Small Square [BEST] Inside

==> 3. Show Line to Divided and R

" [ *** ============ F I N A L - [ F U L L ] ============ ***] "
==========================

In [1]:
import cv2
import numpy as np
import glob
import os
import math

# =============== [ P A R A M E T E R ] - [ F U L L ]
#=========================================================
pathResultImages = r"Result_Images\\"
BEV_Name = "1.Rectangle-BEV Camera Show (Crop).jpg"
heightBEV = 785
widthBEV = 657
# =============== Camera Filename Images
Camera_A_Name = "Camera_A_1.jpg"
Camera_B_Name = "Camera_B_1.jpg"
Camera_C_Name = "Camera_C_1.jpg"
Camera_D_Name = "Camera_D_1.jpg"
print("=> Filename: ", Camera_A_Name,Camera_B_Name,Camera_C_Name,Camera_D_Name)
print()
# =============== Matrix Camera -> BEV
MatrixA = np.zeros((3,3), np.float)
MatrixB = np.zeros((3,3), np.float)
MatrixC = np.zeros((3,3), np.float)
MatrixD = np.zeros((3,3), np.float)
MatrixA[0:3] = [[ 2.36588339e-01, -9.79616203e-01,  2.31036674e+02], [-9.26623812e-03, -1.35976221e+00,  4.85950598e+02], [-6.85560659e-05, -3.12789910e-03,  1.00000000e+00]]
MatrixB[0:3] = [[-2.19189956e+00,  1.08059785e-01,  4.30714845e+02], [-2.98505403e+00, -4.48153295e-01,  6.36091901e+02], [-7.19621378e-03,  4.49285188e-04,  1.00000000e+00]]
MatrixC[0:3] = [[-1.06122502e+00, -5.80395453e-02,  3.81511526e+02], [-1.29407427e+00,  2.14188277e-01,  3.27761048e+02], [-3.18749314e-03, -1.48513790e-04,  1.00000000e+00]]
MatrixD[0:3] = [[-6.56100043e-01, -2.23582382e+00,  4.97126571e+02], [-4.64843971e-02, -2.77641520e+00,  5.24733577e+02], [-1.21516486e-04, -7.20435663e-03,  1.00000000e+00]]
print("=> MatrixA = ", MatrixA)
print("=> MatrixB = ", MatrixB)
print("=> MatrixC = ", MatrixC)
print("=> MatrixD = ", MatrixD)
print("="*50)
print()
# =============== Coordinate Small Square [BEST] Inside    
pointXMin = 225
pointXMax = 411
pointYMin = 290
pointYMax = 519
print("=> pointXMin = ", pointXMin)
print("=> pointXMax = ", pointXMax)
print("=> pointYMin = ", pointYMin)
print("=> pointYMax = ", pointYMax)
print("="*50)
print()
#=========================================================
# =============== [ P A R A M E T E R ] - [ F U L L ]

# =============== [ F U N C T I O N ] - [ F U L L ]
#=========================================================
def readCamera(cameraName, Camera_Filename):
    img = cv2.imread(Camera_Filename)
    if (cameraName == "D"):
        # Rotate D - No Rotate
        pass
    elif (cameraName == "B"):
        # Camera B - Rotate 90 COUNTERCLOCKWISE
        img = cv2.rotate(img, cv2.cv2.ROTATE_90_COUNTERCLOCKWISE)
    elif (cameraName == "C"):
        # Camera C - Rotate 90 CLOCKWISE
        img = cv2.rotate(img, cv2.cv2.ROTATE_90_CLOCKWISE)
    elif (cameraName == "A"):
        # Camera A - Rotate 180 
        img = cv2.rotate(img, cv2.cv2.ROTATE_180)
    cv2.imshow(Camera_Filename, img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    return img
#=========================================================
#>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
def showBEV_Result(img,Camera_Filename,matrix, width,height):
    imgBEV = cv2.warpPerspective(img,matrix, (width,height))
    '''
    cv2.imshow("-BEV-" + Camera_Filename, imgBEV)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    '''
    return imgBEV
#>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
def removeSmallSquareFull(cameraName, imgBEVCamera, xMin,xMax,yMin,yMax):
    thickness = -1
    color = (0,0,0)
    if cameraName == "D":
        img = cv2.rectangle(imgBEVCamera, (0,yMin), (imgBEVCamera.shape[1],yMax), color, thickness)
    elif cameraName == "B":
        img = cv2.rectangle(imgBEVCamera, (xMin,0), (xMax,imgBEVCamera.shape[0]), color, thickness)
    elif cameraName == "C":
        img = cv2.rectangle(imgBEVCamera, (xMin,imgBEVCamera.shape[0]), (xMax,0), color, thickness)
    elif cameraName == "A":
        img = cv2.rectangle(imgBEVCamera, (imgBEVCamera.shape[1],yMin), (0,yMax), color, thickness)
    '''
    cv2.imshow("removeSmallSquareFull", img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    '''
    return img
#>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
def showLineDividedRemove(cameraName,imgBEVCamera,BEV_Camera_Name, heightBEV,widthBEV):
    color = (0,0,255)
    thickness = 1
    # Divided Into 2 Parts
    halfHeight = int(heightBEV/2)
    halfWidth = int(widthBEV/2)
    if cameraName == "A" or cameraName == "D":
        imgBEVCamera = cv2.line(imgBEVCamera, (0,halfHeight), (widthBEV,halfHeight), color, thickness)
        if cameraName == "D":
            # Removes Buttom Parts
            imgBEVCamera = cv2.rectangle(imgBEVCamera, (0,halfHeight), (widthBEV,halfHeight*2), (0,0,0), -1)
        elif cameraName == "A":
            # Removes Top Parts
            imgBEVCamera = cv2.rectangle(imgBEVCamera, (0,0), (widthBEV,halfHeight), (0,0,0), -1)
    elif cameraName == "C" or cameraName == "B":
        imgBEVCamera = cv2.line(imgBEVCamera, (halfWidth,0), (halfWidth,heightBEV), color, thickness)
        if cameraName == "C":
            # Removes Left Parts
            imgBEVCamera = cv2.rectangle(imgBEVCamera, (0,0), (halfWidth,heightBEV), (0,0,0), -1)
        elif cameraName == "B":
            # Removes Right Parts
            imgBEVCamera = cv2.rectangle(imgBEVCamera, (halfWidth,0), (halfWidth*2,heightBEV), (0,0,0), -1)
    '''
    cv2.imshow("Remove Parts" + BEV_Camera_Name, imgBEVCamera)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    '''
    return imgBEVCamera
#>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
def cropBEVRemovaCamera(cameraName,img,BEVRemove_Camera_Name):
    height = img.shape[0]
    width = img.shape[1]
    if cameraName == "D":
        x = 0
        h = int(height/2)
        y = 0
        w = width
        img = img[y:y+h, x:x+w]
        imgRotate = img
    elif cameraName == "B":
        x = 0
        h = height
        y = 0
        w = int(width/2)
        img = img[y:y+h, x:x+w]
        # Rotate 
        imgRotate = cv2.rotate(img, cv2.ROTATE_90_CLOCKWISE)
    elif cameraName == "C":
        x = int(width/2)
        h = height
        y = 0
        w = int(width/2)
        img = img[y:y+h, x:x+w]
        imgRotate = cv2.rotate(img, cv2.ROTATE_90_COUNTERCLOCKWISE)
    elif cameraName == "A":
        x = 0
        h = int(height/2)
        y = int(height/2)
        w = width
        img = img[y:y+h, x:x+w]
        imgRotate = cv2.rotate(img, cv2.ROTATE_180)
    cv2.imshow("-Crop-" + BEVRemove_Camera_Name, img)
    cv2.imshow("-Rotate-" + BEVRemove_Camera_Name, imgRotate)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    return img, imgRotate
#>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
#=========================================================
def writeNewImage(img, pathWriteImg):
    cv2.imwrite(pathWriteImg, img)
#=========================================================
# =============== [ F U N C T I O N ] - [ F U L L ]

# =============== [ M A I N - F U N C T I O N ] - [ F U L L ]
#=========================================================
def main():
    print("==> 1. Size BEV")
    print("Height: ", heightBEV)
    print("Width: ", widthBEV)
    print()

    print("==> 2. Read Camera & Rotate")
    print()
    # ==================== Read Camera & Rotate
    imgCameraA = readCamera("A", Camera_A_Name)
    imgCameraB = readCamera("B", Camera_B_Name)
    imgCameraC = readCamera("C", Camera_C_Name)
    imgCameraD = readCamera("D", Camera_D_Name)

    print("==> 2. Show  BEV Result")
    print()
    # Show  BEV Result
    imgCameraA = showBEV_Result(imgCameraA,Camera_A_Name,MatrixA, widthBEV,heightBEV)
    imgCameraB = showBEV_Result(imgCameraB,Camera_B_Name,MatrixB, widthBEV,heightBEV)
    imgCameraC = showBEV_Result(imgCameraC,Camera_C_Name,MatrixC, widthBEV,heightBEV)
    imgCameraD = showBEV_Result(imgCameraD,Camera_D_Name,MatrixD, widthBEV,heightBEV)

    print("==> 3. Remove Small Square [BEST] Inside - [ F U L L ]")
    print()
    # Remove Small Square [BEST] Inside - [ F U L L ]
    imgCameraA = removeSmallSquareFull("A", imgCameraA, pointXMin,pointXMax,pointYMin,pointYMax)
    imgCameraB = removeSmallSquareFull("B", imgCameraB, pointXMin,pointXMax,pointYMin,pointYMax)
    imgCameraC = removeSmallSquareFull("C", imgCameraC, pointXMin,pointXMax,pointYMin,pointYMax)
    imgCameraD = removeSmallSquareFull("D", imgCameraD, pointXMin,pointXMax,pointYMin,pointYMax)

    print("==> 3. Show Line to Divided and Removes")
    print()
    # Show Line to Divided and Removes
    imgCameraA = showLineDividedRemove("A",imgCameraA,Camera_A_Name, heightBEV,widthBEV)
    imgCameraB = showLineDividedRemove("B",imgCameraB,Camera_B_Name, heightBEV,widthBEV)
    imgCameraC = showLineDividedRemove("C",imgCameraC,Camera_C_Name, heightBEV,widthBEV)
    imgCameraD = showLineDividedRemove("D",imgCameraD,Camera_D_Name, heightBEV,widthBEV)

    print("==> 4. Show Line to Divided and Removes")
    print()
    # Read BEV Remove Camera & Crop
    imgCameraA, imgCameraRotateA = cropBEVRemovaCamera("A", imgCameraA,Camera_A_Name)
    imgCameraB, imgCameraRotateB = cropBEVRemovaCamera("B", imgCameraB,Camera_B_Name)
    imgCameraC, imgCameraRotateC = cropBEVRemovaCamera("C", imgCameraC,Camera_C_Name)
    imgCameraD, imgCameraRotateD = cropBEVRemovaCamera("D", imgCameraD,Camera_D_Name)

    print("==> 4. Write new Images - [ F U L L ]")
    print()
    # Write new Images
    filenameImage = "5.1.BEV-FINAL-FULL-"
    filenameImage = pathResultImages + filenameImage
    writeNewImage(imgCameraA, filenameImage+Camera_A_Name)
    writeNewImage(imgCameraB, filenameImage+Camera_B_Name)
    writeNewImage(imgCameraC, filenameImage+Camera_C_Name)
    writeNewImage(imgCameraD, filenameImage+Camera_D_Name)
    # Write new Images Rotate
    filenameImage = "5.2.R.BEV-FINAL-FULL-"
    filenameImage = pathResultImages + filenameImage
    writeNewImage(imgCameraRotateA, filenameImage+Camera_A_Name)
    writeNewImage(imgCameraRotateB, filenameImage+Camera_B_Name)
    writeNewImage(imgCameraRotateC, filenameImage+Camera_C_Name)
    writeNewImage(imgCameraRotateD, filenameImage+Camera_D_Name)
#=========================================================
# =============== [ M A I N - F U N C T I O N ] - [ F U L L ]

if __name__ == '__main__':       
    # Calling main() function 
    main()

=> Filename:  Camera_A_1.jpg Camera_B_1.jpg Camera_C_1.jpg Camera_D_1.jpg

=> MatrixA =  [[-3.97331246e-01 -3.35970227e+00  7.07705644e+02]
 [ 2.13662752e-02 -1.88831907e+00  3.86122305e+02]
 [ 1.58521856e-05 -5.91162509e-03  1.00000000e+00]]
=> MatrixB =  [[-3.70282162e+00  1.35447487e-01  6.77917664e+02]
 [-2.42284350e+00 -3.40403340e-01  5.08361784e+02]
 [-6.71155256e-03  3.64932494e-04  1.00000000e+00]]
=> MatrixC =  [[-1.76856484e+00 -4.01966105e-03  6.30668332e+02]
 [-1.06914569e+00  2.12590700e-01  2.83023659e+02]
 [-3.03334633e-03 -1.59576052e-05  1.00000000e+00]]
=> MatrixD =  [[ 2.93275532e-01 -1.75818320e+00  4.90120234e+02]
 [ 1.44451663e-02 -1.17911738e+00  4.31525305e+02]
 [ 6.99898154e-05 -3.08305253e-03  1.00000000e+00]]

=> pointXMin =  450
=> pointXMax =  655
=> pointYMin =  249
=> pointYMax =  438

==> 1. Size BEV
Height:  706
Width:  1040

==> 2. Read Camera & Rotate

==> 2. Show  BEV Result

==> 3. Remove Small Square [BEST] Inside - [ F U L L ]

==> 3. Show Line t